In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time


# Data Consumption

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@dbxdev.dfs.core.windows.net/customers")
display(df)

In [0]:
df = df.drop('_rescued_data')
display(df)

In [0]:
df = df.withColumn("domains", split(col('email'), '@')[1])
display(df)

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending=False).display()


In [0]:
df_gmail = df.filter(col("domains") == "gmail.com")
display(df_gmail)
time.sleep(5)

df_yahoo = df.filter(col("domains") == "yahoo.com")
display(df_yahoo)
time.sleep(5)

df_hotmail = df.filter(col("domains") == "hotmail.com")
display(df_hotmail)
time.sleep(5)


In [0]:
df = df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df = df.drop("first_name","last_name")
display(df)


# Data Persistence

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@dbxdev.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS azuredb_catalog.silver.customers_silver
USING DELTA 
LOCATION 'abfss://silver@dbxdev.dfs.core.windows.net/customers'

In [0]:
%sql
SELECT * FROM azuredb_catalog.silver.customers_silver
